In [18]:
pwd
cd myworkspace/mysite
ls

/home/chris/home/workspace/Python3/Level 4/Django/myworkspace/mysite
bash: cd: myworkspace/mysite: No such file or directory
db.sqlite3  manage.py  mysite  polls


Restart the server

In [19]:
fuser -k 7000/tcp
python manage.py runserver 7000 &

7000/tcp:            491537
[3] 491605
[2]   Exit 247                python manage.py runserver 7000


Django has an interactive Python shell which we can use to add data to sqlite.  Again, we'll use Unix hereis documents to controll the shell.  Of course, in reality, you would just use the command line.

In [73]:
python manage.py shell << EOF &
from polls.models import Choice, Question  # Import the model classes we just wrote.
print(Question.objects.all())
EOF

[4] 492679


We can now create a new Question:

In [74]:
python manage.py shell << EOF
from polls.models import Choice, Question  # Import the model classes we just wrote.
from django.utils import timezone
q = Question(question_text="What's new?", pub_date=timezone.now())

# Save the object into the database. You have to call save() explicitly.
q.save()
print(q.id)
EOF

<QuerySet [<Question: What's new?>, <Question: What's up?>, <Question: What's up?>, <Question: What's new?>, <Question: What's up?>, <Question: What's up?>, <Question: What's up?>, <Question: What's up 2?>, <Question: What's up 2?>, <Question: What's up 2?>, <Question: What's up 2?>, <Question: What's up 2?>, <Question: What's up 2?>, <Question: What's up 2?>]>
15
[4]+  Done                    python manage.py shell <<EOF
from polls.models import Choice, Question  # Import the model classes we just wrote.
print(Question.objects.all())
EOF



We can inspect the question text and publication date with a slightly modified script:

In [87]:
# Access model field values via Python attributes.
python manage.py shell << EOF
from polls.models import Choice, Question  # Import the model classes we just wrote.
from django.utils import timezone
q = Question(question_text="What's new?", pub_date=timezone.now())

#print(q.question_text)
#print(q.pub_date)

# Change values by changing the attributes, then calling save().
#q.question_text = "What's up 2?"
q.save()

# objects.all() displays all the questions in the database.
print(Question.objects.all())
#z = Question.objects.all()[1]
#print(z.question_text)
#print(z.pub_date)
EOF

<QuerySet [<Question: Question object (16)>, <Question: Question object (17)>]>


That's a bit cryptic.  We can improve the print out by changing the model slightly.
The current model looks like:

In [86]:
cat polls/models.py

/home/chris/home/workspace/Python3/Level 4/Django/myworkspace/mysite/polls/models.py changed, reloading.
Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
September 05, 2021 - 20:16:57
Django version 3.2.4, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.
cat polls/models.py^Jfrom django.db import models
from django.utils import timezone
import datetime

class Question(models.Model):
    def was_published_recently(self):
         return self.pub_date >= timezone.now() - datetime.timedelta(days=1)

    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')


class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)


Change the `Question` and `Choice` classes by adding `__str__` methods:

In [89]:
cat << EOF > polls/models.py
from django.db import models
from django.utils import timezone
import datetime

class Question(models.Model):
    def __str__(self):
         return self.question_text
         
    def was_published_recently(self):
         return self.pub_date >= timezone.now() - datetime.timedelta(days=1)

    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')


class Choice(models.Model):
    def __str__(self):
         return self.choice_text

    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)
EOF

#python manage.py migrate

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
September 05, 2021 - 20:17:27
Django version 3.2.4, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.
cat << EOF > polls/models.py^J

Now run the query again (to see text printed):

In [91]:
# Access model field values via Python attributes.
python manage.py shell << EOF
from polls.models import Choice, Question  # Import the model classes we just wrote.
from django.utils import timezone
#q = Question(question_text="What's new?", pub_date=timezone.now())

#print(q.question_text)
#print(q.pub_date)

# Change values by changing the attributes, then calling save().
#q.question_text = "What's up 2?"
#q.save()

# objects.all() displays all the questions in the database.
print(Question.objects.all())
#z = Question.objects.all()[1]
#print(z.question_text)
#print(z.pub_date)
EOF

<QuerySet [<Question: What's new?>, <Question: What's new?>]>


In [ ]:
Other things you can do in the shell:

In [106]:
python manage.py shell << EOF
from polls.models import Choice, Question
from django.utils import timezone
print(Question.objects.all())
q = Question.objects.all()[0]
print(q.question_text)
print(q.pub_date)
print(q.id)
print(q.was_published_recently())

print(Question.objects.filter(id=q.id))

EOF

<QuerySet [<Question: What's new?>, <Question: What's new?>]>
What's new?
2021-09-05 20:06:24.163901+00:00
16
True
<QuerySet [<Question: What's new?>]>


Give the `Question` a couple of `Choices`.  At present there are no choices:

In [107]:
python manage.py shell << EOF
from polls.models import Choice, Question
q = Question.objects.all()[0]
print(q.choice_set.all())
EOF

<QuerySet []>


Use `create` to create choices:

In [110]:
python manage.py shell << EOF
from polls.models import Choice, Question
# Create three choices.
q = Question.objects.all()[0]
q.choice_set.create(choice_text='Not much', votes=0)
q.choice_set.create(choice_text='The sky', votes=0)
c = q.choice_set.create(choice_text='Just hacking again', votes=0)
print(f"original question: {c.question}")
print(f"choices: {q.choice_set.all()}")
print(f"count of choices: {q.choice_set.count()}")
EOF

original question: What's new?
choices: <QuerySet [<Choice: Not much>, <Choice: The sky>, <Choice: Just hacking again>, <Choice: Not much>, <Choice: The sky>, <Choice: Just hacking again>, <Choice: Not much>, <Choice: The sky>, <Choice: Just hacking again>]>
count of choices: 9


The API automatically follows relationships as far as you need.
Use double underscores to separate relationships.
This works as many levels deep as you want; there's no limit.

Find all Choices for any question whose pub_date is in this year
(reusing the 'current_year' variable we created above).

In [114]:
python manage.py shell << EOF
from polls.models import Choice, Question
from django.utils import timezone
current_year = timezone.now().year
print(Choice.objects.filter(question__pub_date__year=current_year))


q = Question.objects.all()[0]
c = q.choice_set.filter(choice_text__startswith='Just hacking')
print(q.choice_set.all())
EOF

<QuerySet [<Choice: Not much>, <Choice: The sky>, <Choice: Not much>, <Choice: The sky>, <Choice: Not much>, <Choice: The sky>]>
<QuerySet [<Choice: Not much>, <Choice: The sky>, <Choice: Not much>, <Choice: The sky>, <Choice: Not much>, <Choice: The sky>]>


Let's delete one of the choices. Use delete() for that.

In [115]:
python manage.py shell << EOF
from polls.models import Choice, Question
from django.utils import timezone
current_year = timezone.now().year
q = Question.objects.all()[0]
c = q.choice_set.filter(choice_text__startswith='Just hacking')
c.delete()
print(q.choice_set.all())
EOF

<QuerySet [<Choice: Not much>, <Choice: The sky>, <Choice: Not much>, <Choice: The sky>, <Choice: Not much>, <Choice: The sky>]>


We can use a webapp to see all of the above.  First create superuser for Django Admin by
running the following command in xterm:

In [116]:
xterm -fg black -bg white -fa 'Monospace' -fs 14 -e 'python manage.py createsuperuser' &

[4] 494471


Register the `Question` with the app:

In [120]:
cat << EOF > polls/admin.py
from django.contrib import admin
from .models import Question
admin.site.register(Question)

EOF

Check the registration worked:

In [124]:
cat polls/admin.py

from django.contrib import admin
from .models import Question
admin.site.register(Question)



Now fire up the admin app and login as the superuser.  You can now investigate all the `Questions` and `Choices`.

In [125]:
firefox http://127.0.0.1:7000/admin/ &

[4] 494715
